In [1]:
import main as sugar

In [2]:
N = 20
size = 99
vision = 1
tax_brackets = [0,0]
tax_percentages = [0,0]
inheritance_tax_brackets = [0, 10, 30, 50, 100]
inheritance_tax_percentages = [0, 0.3, 0.3, 0.35, 0.6]
starting_wealth = 5
steps = 5

parameters = N, size, vision, tax_brackets, tax_percentages, inheritance_tax_brackets, inheritance_tax_percentages, starting_wealth, steps

df = sugar.main(parameters)
df

saved data for 2021-01-20 17.01


Wealth  Position  Vision  N_Agents  Size tax brackets  \
Step AgentID                                                          
0    0           5.0  (93, 56)       1        20    99       [0, 0]   
     1           5.0  (42, 91)       1        20    99       [0, 0]   
     2           5.0  (49, 31)       1        20    99       [0, 0]   
     3           5.0  (57, 19)       1        20    99       [0, 0]   
     4           5.0  (46, 25)       1        20    99       [0, 0]   
...              ...       ...     ...       ...   ...          ...   
4    15         50.0  (54, 38)       1        20    99       [0, 0]   
     16          1.0  (30, 22)       1        20    99       [0, 0]   
     17         17.0  (88, 85)       1        20    99       [0, 0]   
     18          1.0  (90, 94)       1        20    99       [0, 0]   
     19          1.0  (15, 27)       1        20    99       [0, 0]   

             tax percentages inheritance taxbrackets  \
Step AgentID                                           
0    0                [0, 0]    [0, 10, 30, 50, 100]   
     1                [0, 0]    [0, 10, 30, 50, 100]   
     2                [0, 0]    [0, 10, 30, 50, 100]   
     3                [0, 0]    [0, 10, 30, 50, 100]   
     4                [0, 0]    [0, 10, 30, 50, 100]   
...                      ...                     ...   
4    15               [0, 0]    [0, 10, 30, 50, 100]   
     16               [0, 0]    [0, 10, 30, 50, 100]   
     17               [0, 0]    [0, 10, 30, 50, 100]   
     18               [0, 0]    [0, 10, 30, 50, 100]   
     19               [0, 0]    [0, 10, 30, 50, 100]   

             inheritance tax percentages  Starting Wealth  
Step AgentID                                               
0    0          [0, 0.3, 0.3, 0.35, 0.6]                5  
     1          [0, 0.3, 0.3, 0.35, 0.6]                5  
     2          [0, 0.3, 0.3, 0.35, 0.6]                5  
     3          [0, 0.3, 0.3, 0.35, 0.6]                5  
     4          [0, 0.3, 0.3, 0.35, 0.6]                5  
...                                  ...              ...  
4    15         [0, 0.3, 0.3, 0.35, 0.6]                5  
     16         [0, 0.3, 0.3, 0.35, 0.6]                5  
     17         [0, 0.3, 0.3, 0.35, 0.6]                5  
     18         [0, 0.3, 0.3, 0.35, 0.6]                5  
     19         [0, 0.3, 0.3, 0.35, 0.6]                5  

[100 rows x 10 columns]

In [17]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

# Define the model inputs
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359]]
}

# Generate samples
param_values = saltelli.sample(problem, 1000)

print(len(param_values))

# Run model (example)
Y = Ishigami.evaluate(param_values)

print(len(Y))

# Perform analysis
Si = sobol.analyze(problem, Y, print_to_console=True)

# Print the first-order sensitivity indices
print(Si['S1'])

8000
8000
Parameter S1 S1_conf ST ST_conf
x1 0.307975 0.056343 0.560137 0.091265
x2 0.447767 0.050136 0.438722 0.037017
x3 -0.004255 0.053589 0.242845 0.023711

Parameter_1 Parameter_2 S2 S2_conf
x1 x2 0.012205 0.084418
x1 x3 0.251526 0.104773
x2 x3 -0.009954 0.067703
[ 0.30797549  0.44776661 -0.00425452]
